In [1]:
import pandas as pd
import numpy as np
import joblib
import warnings
warnings.filterwarnings('ignore')

# Importar transformadores personalizados (necesario para cargar el pickle)
import os, sys
sys.path.append(r"C:\Users\hsuna\Desktop\Proyecto Final Product Development\proyecto final v2\repo-seriestemporales-g4-pd\notebooks")

from custom_transformers import (
    TemporalFeatures, LagCreator, OutlierTreatment, 
    LogTransformation, CategoricalEncoder, ToNumericTransformer
)

print("="*80)
print("GENERACIÓN DE PREDICCIONES - INFERENCIA")
print("="*80)

# ==================================================================================
# CARGAR PIPELINE COMPLETO
# ==================================================================================

print("\nCargando pipeline completo...")
try:
    complete_pipeline = joblib.load(r'C:\Users\hsuna\Desktop\Proyecto Final Product Development\proyecto final v2\repo-seriestemporales-g4-pd\models\ventas_complete_pipeline.pkl')
    print("✓ Pipeline cargado exitosamente")
except Exception as e:
    print(f"✗ Error al cargar pipeline: {str(e)}")
    raise

# ==================================================================================
# CARGAR DATOS DE TESTING
# ==================================================================================

print("\n" + "="*80)
print("CARGANDO DATOS DE TESTING")
print("="*80)

# Cargar datos crudos completos
df_completo = pd.read_csv(r'C:\Users\hsuna\Desktop\Proyecto Final Product Development\proyecto final v2\repo-seriestemporales-g4-pd\data\raw\ventas.csv')

# Convertir y ordenar por fecha
df_completo['Fecha_Venta'] = pd.to_datetime(df_completo['Fecha_Venta'], dayfirst=True, errors='coerce', format='mixed')
df_completo = df_completo.sort_values(['Codigo_Sucursal', 'Codigo_Producto', 'Fecha_Venta'])
df_completo = df_completo.reset_index(drop=True)

# Usar el último 20% como testing (datos que el modelo no vio)
split_idx = int(len(df_completo) * 0.8)
test_data = df_completo.iloc[split_idx:].copy()

print(f"✓ Registros en testing: {len(test_data):,}")
print(f"✓ Período: {test_data['Fecha_Venta'].min()} a {test_data['Fecha_Venta'].max()}")

# Preparar datos
X_test = test_data.drop('Total', axis=1) if 'Total' in test_data.columns else test_data.copy()
y_test = test_data['Total'].values if 'Total' in test_data.columns else None

print(f"✓ Shape de X_test: {X_test.shape}")
print(f"✓ Columnas en X_test: {list(X_test.columns)}")

# ==================================================================================
# GENERAR PREDICCIONES
# ==================================================================================

print("\n" + "="*80)
print("GENERANDO PREDICCIONES")
print("="*80)

try:
    predictions = complete_pipeline.predict(X_test)
    print(f"✓ Predicciones generadas: {len(predictions):,}")
    
    # Estadísticas de las predicciones
    print("\n" + "-"*80)
    print("ESTADÍSTICAS DE LAS PREDICCIONES")
    print("-"*80)
    print(f"Predicción mínima:    Q{predictions.min():>12,.2f}")
    print(f"Predicción máxima:    Q{predictions.max():>12,.2f}")
    print(f"Predicción promedio:  Q{predictions.mean():>12,.2f}")
    print(f"Mediana:              Q{np.median(predictions):>12,.2f}")
    print(f"Desviación estándar:  Q{predictions.std():>12,.2f}")
    
    # Si tenemos valores reales, calcular métricas
    if y_test is not None:
        from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
        
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        mae = mean_absolute_error(y_test, predictions)
        r2 = r2_score(y_test, predictions)
        mape = np.mean(np.abs((y_test - predictions) / y_test)) * 100
        
        print("\n" + "-"*80)
        print("MÉTRICAS DE EVALUACIÓN EN TESTING")
        print("-"*80)
        print(f"RMSE (Root Mean Squared Error):  Q{rmse:>12,.2f}")
        print(f"MAE (Mean Absolute Error):        Q{mae:>12,.2f}")
        print(f"R² (Coeficiente de determinación): {r2:>11.4f}")
        print(f"MAPE (Mean Absolute % Error):      {mape:>11.2f}%")
    
except Exception as e:
    print(f"✗ Error al generar predicciones: {str(e)}")
    import traceback
    traceback.print_exc()
    raise

# ==================================================================================
# CREAR DATAFRAME CON RESULTADOS
# ==================================================================================

print("\n" + "="*80)
print("PREPARANDO RESULTADOS")
print("="*80)

# Crear DataFrame con resultados
results_df = test_data[['Codigo_Sucursal', 'Fecha_Venta', 'Codigo_Producto', 'Unidades_Vendidas']].copy()
results_df['Prediccion_Total'] = predictions

if y_test is not None:
    results_df['Total_Real'] = y_test
    results_df['Error_Absoluto'] = np.abs(results_df['Total_Real'] - results_df['Prediccion_Total'])
    results_df['Error_Porcentual'] = (results_df['Error_Absoluto'] / results_df['Total_Real'] * 100).round(2)

print("\n✓ Primeras 10 predicciones:")
print(results_df.head(10).to_string(index=False))

# ==================================================================================
# GUARDAR PREDICCIONES
# ==================================================================================

print("\n" + "="*80)
print("GUARDANDO PREDICCIONES")
print("="*80)

output_path = r'C:\Users\hsuna\Desktop\Proyecto Final Product Development\proyecto final v2\repo-seriestemporales-g4-pd\data\processed\predicciones_ventas.csv'
results_df.to_csv(output_path, index=False)

print(f"✓ Archivo guardado en:")
print(f"  {output_path}")
print(f"✓ Total de predicciones: {len(results_df):,}")

# ==================================================================================
# ANÁLISIS AGREGADO
# ==================================================================================

print("\n" + "="*80)
print("ANÁLISIS AGREGADO DE PREDICCIONES")
print("="*80)

# Resumen por sucursal
print("\n" + "-"*80)
print("RESUMEN POR SUCURSAL")
print("-"*80)
summary_sucursal = results_df.groupby('Codigo_Sucursal').agg({
    'Prediccion_Total': ['sum', 'mean', 'count']
}).round(2)
summary_sucursal.columns = ['Total_Predicho', 'Promedio', 'Num_Registros']
summary_sucursal['Total_Predicho'] = summary_sucursal['Total_Predicho'].apply(lambda x: f"Q{x:,.2f}")
summary_sucursal['Promedio'] = summary_sucursal['Promedio'].apply(lambda x: f"Q{x:,.2f}")
print(summary_sucursal)

# Resumen por producto (Top 10)
print("\n" + "-"*80)
print("RESUMEN POR PRODUCTO (TOP 10)")
print("-"*80)
summary_producto = results_df.groupby('Codigo_Producto').agg({
    'Prediccion_Total': ['sum', 'mean', 'count']
}).round(2)
summary_producto.columns = ['Total_Predicho', 'Promedio', 'Num_Registros']
summary_producto = summary_producto.sort_values('Total_Predicho', ascending=False).head(10)
summary_producto['Total_Predicho'] = summary_producto['Total_Predicho'].apply(lambda x: f"Q{x:,.2f}")
summary_producto['Promedio'] = summary_producto['Promedio'].apply(lambda x: f"Q{x:,.2f}")
print(summary_producto)

# Resumen temporal (por mes)
print("\n" + "-"*80)
print("RESUMEN TEMPORAL (POR MES)")
print("-"*80)
results_df['Año_Mes'] = results_df['Fecha_Venta'].dt.to_period('M')
summary_temporal = results_df.groupby('Año_Mes').agg({
    'Prediccion_Total': ['sum', 'mean', 'count']
}).round(2)
summary_temporal.columns = ['Total_Predicho', 'Promedio', 'Num_Registros']
summary_temporal['Total_Predicho'] = summary_temporal['Total_Predicho'].apply(lambda x: f"Q{x:,.2f}")
summary_temporal['Promedio'] = summary_temporal['Promedio'].apply(lambda x: f"Q{x:,.2f}")
print(summary_temporal.tail(6))  # Últimos 6 meses

# ==================================================================================
# RESUMEN FINAL
# ==================================================================================

print("\n" + "="*80)
print("RESUMEN DEL PROCESO DE INFERENCIA")
print("="*80)

total_predicho = results_df['Prediccion_Total'].sum()
print(f"""
✓ Registros procesados: {len(results_df):,}
✓ Total predicho: Q{total_predicho:,.2f}
✓ Promedio por transacción: Q{results_df['Prediccion_Total'].mean():,.2f}
✓ Sucursales únicas: {results_df['Codigo_Sucursal'].nunique()}
✓ Productos únicos: {results_df['Codigo_Producto'].nunique()}
✓ Archivo de predicciones guardado exitosamente
""")

if y_test is not None:
    print(f"✓ Métricas en testing:")
    print(f"  - RMSE: Q{rmse:,.2f}")
    print(f"  - R²: {r2:.4f}")
    print(f"  - MAPE: {mape:.2f}%")

print("\n" + "="*80)
print("PROCESO DE INFERENCIA COMPLETADO EXITOSAMENTE")
print("="*80)

GENERACIÓN DE PREDICCIONES - INFERENCIA

Cargando pipeline completo...
✓ Pipeline cargado exitosamente

CARGANDO DATOS DE TESTING
✓ Registros en testing: 14,631
✓ Período: 2020-01-02 00:00:00 a 2025-10-22 00:00:00
✓ Shape de X_test: (14631, 4)
✓ Columnas en X_test: ['Codigo_Sucursal', 'Fecha_Venta', 'Codigo_Producto', 'Unidades_Vendidas']

GENERANDO PREDICCIONES
Advertencia: 9821 valores no vistos en 'Codigo_Sucursal' - usando valor default
✓ Predicciones generadas: 14,631

--------------------------------------------------------------------------------
ESTADÍSTICAS DE LAS PREDICCIONES
--------------------------------------------------------------------------------
Predicción mínima:    Q        0.82
Predicción máxima:    Q   34,016.51
Predicción promedio:  Q      481.66
Mediana:              Q      129.31
Desviación estándar:  Q    1,300.74

--------------------------------------------------------------------------------
MÉTRICAS DE EVALUACIÓN EN TESTING
------------------------------